In [1]:
import numpy as np
import pandas as pd
import os

from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, TrainingArguments, Trainer, ModernBertForSequenceClassification
from datasets import Dataset

import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    
    # print(f"Avg pred: {sum(predictions)/len(predictions)}, Avg Labels: {sum(labels)/len(labels)}")
    
    return metric.compute(predictions=predictions, references=labels)

df = pd.read_csv( "../data/pairwise-model-data/pairedwise-tweets.tsv", sep = '\t')
df = pd.DataFrame(df)

dataset = Dataset.from_pandas(df)
dataset.shuffle()
split = dataset.train_test_split(test_size=0.2)
train_dataset = split["train"].remove_columns('Unnamed: 0')
eval_dataset = split["test"].remove_columns('Unnamed: 0')

Traceback (most recent call last):
  File "/home/guilherme/workspace/feup-ir-project/.ir-env/lib/python3.12/site-packages/torch/_inductor/compile_worker/__main__.py", line 10, in <module>
    from torch._inductor.async_compile import pre_fork_setup
  File "/home/guilherme/workspace/feup-ir-project/.ir-env/lib/python3.12/site-packages/torch/__init__.py", line 2611, in <module>
    from torch import _meta_registrations
  File "/home/guilherme/workspace/feup-ir-project/.ir-env/lib/python3.12/site-packages/torch/_meta_registrations.py", line 12, in <module>
    from torch._decomp import (
  File "/home/guilherme/workspace/feup-ir-project/.ir-env/lib/python3.12/site-packages/torch/_decomp/__init__.py", line 276, in <module>
    import torch._decomp.decompositions
  File "/home/guilherme/workspace/feup-ir-project/.ir-env/lib/python3.12/site-packages/torch/_decomp/decompositions.py", line 16, in <module>
    import torch._prims as prims
  File "/home/guilherme/workspace/feup-ir-project/.ir-en

KeyboardInterrupt: 

In [ ]:
def preprocess_function(examples):
    full_str = "[CLS] "
    full_str += examples["query"]
    
    for key in ["paper1", "paper2"]:
        if examples[key] != None:
            full_str += " [SEP] " + examples[key]
    
    return tokenizer(full_str, truncation=True).to('cuda')


id2label = {0: "0", 1: "1"}
label2id = {"0": 0, "1": 1}

tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")
model = ModernBertForSequenceClassification.from_pretrained("answerdotai/ModernBERT-base", num_labels=2, id2label=id2label, label2id=label2id)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataset_tokenized = train_dataset.map(preprocess_function)

Map:   0%|          | 0/61350 [00:00<?, ? examples/s]

In [ ]:
test_dataset_tokenized = eval_dataset.map(preprocess_function)

Map:   0%|          | 0/15338 [00:00<?, ? examples/s]

In [ ]:

training_args = TrainingArguments(
    output_dir="../models/pairwise-classifier-large",
    learning_rate=2e-6,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=16000,
    save_strategy="steps",
    save_steps=16000,
    load_best_model_at_end=True,
    push_to_hub=False,
    fp16=True,
    optim="adamw_torch",
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=test_dataset_tokenized,
    compute_metrics=compute_metrics
)

# trainer.evaluate()
trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss


KeyboardInterrupt: 